# CNN Working Details

In [10]:
# Imports

import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import torch 
import torch.nn as nn 
from torch.utils.data import TensorDataset, DataLoader, Dataset
from torch.optim import SGD, Adam 
from torchvision import datasets 

from torchsummary import summary

# Device
device = 'cuda' if torch.cuda.is_available() else 'cpu' 

In [8]:
# Data

# PyTorch Input Format: batch_size * no_of_channels * height * width
X_train = torch.tensor(
    [
        [
            [
                [1,2,3,4],
                [2,3,4,5],
                [5,6,7,8],
                [1,3,4,5]
                ]
            ],
        [
            [
                [-1,2,3,-4],
                [2,-3,4,5],
                [-5,6,-7,8],
                [-1,-3,-4,-5]
                ]
            ]
        ]
    ).to(device).float()

X_train /= 8    # Scale the data
y_train = torch.tensor([0,1]).to(device).float()

print(X_train.shape)

torch.Size([2, 1, 4, 4])


In [31]:
# Model

def get_model():

    model = nn.Sequential(
        nn.Conv2d(
            in_channels=1,
            out_channels=1,
            kernel_size=3
        ),
        nn.MaxPool2d(
            kernel_size=2
        ),
        nn.ReLU(),
        nn.Linear(1, 1),
        nn.Sigmoid()
    ).to(device)

    loss_fn = nn.BCELoss()

    optimizer = Adam(
        model.parameters(),
        lr=1e-3
        )

    return model, loss_fn, optimizer

In [32]:
# Training Function

def train_batch(x, y, model, loss_fn, optimizer):
    model.train()
    predictions = model(x)
    batch_loss = loss_fn(predictions.squeeze(0), y)
    batch_loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    return batch_loss.item()


model, loss_fn, optimizer = get_model()

# DataLoader

train_dl = DataLoader(
    TensorDataset(
        X_train, 
        y_train 
    )
)


In [ ]:
# Train Model

for epoch in range(2000):
    for index, batch in enumerate(iter(train_dl)):
        x, y = batch 
        batch_loss = train_batch(x, y, model, loss_fn, optimizer)

## Forward Propagating output

In [36]:
list(model.children())

[Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1)),
 MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
 ReLU(),
 Linear(in_features=1, out_features=1, bias=True),
 Sigmoid()]

In [39]:
# Extract layers which have weight attribute

(cnn_w, cnn_b), (lin_w, lin_b) = [
    (
        layer.weight.data, 
        layer.bias.data
    ) 
    for layer in list(model.children()) if hasattr(layer, 'weight')
]

In [41]:
# Convolution

h_im, w_im = X_train.shape[2:]
h_conv, w_conv = cnn_w.shape[2:]
sumprod = torch.zeros((h_im - h_conv + 1, w_im - w_conv + 1))

for i in range(h_im - h_conv + 1):
    for j in range(w_im - w_conv + 1):
        img_subset = X_train[0, 0, i:(i+3), j:(j+3)]
        model_filter = cnn_w.reshape(3,3)
        val = torch.sum(img_subset*model_filter) + cnn_b
        sumprod[i,j] = val

In [46]:
# ReLU
sumprod.clamp_min_(0)

# MaxPool
pooling_layer_output = torch.max(sumprod)

# Preceding Output
intermediate_output_value = pooling_layer_output * lin_w + lin_b

# Sigmoid
print(torch.sigmoid(intermediate_output_value))

tensor([[0.4615]], device='cuda:0')
